In [11]:
# import gdown

# # URL của tệp trong Google Drive, bạn cần thay thế phần "file_id" bằng ID của tệp bạn muốn tải

# url = "https://drive.google.com/drive/folders/16fZtsa1CfuH-YmPJUzrLEiLxhvlCi-GA?usp=drive_link"

# # Tải tệp về
# gdown.download(url, "social_network.graphml", quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/16fZtsa1CfuH-YmPJUzrLEiLxhvlCi-GA?usp=drive_link
To: /content/social_network.graphml
1.22MB [00:00, 56.9MB/s]


'social_network.graphml'

# Không sử dụng code tải drive trên


In [31]:
import networkx as nx
import matplotlib.pyplot as plt

# Load the GraphML file
graph_file_path = 'social_network.graphml'


G = nx.read_graphml(graph_file_path)

# # Draw the graph
# plt.figure(figsize=(8, 8))
# nx.draw(G, with_labels=True, node_color='lightblue', font_weight='bold', node_size=3000, font_size=10, edge_color='gray')
# plt.show()
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

(num_nodes, num_edges)

(41547, 95393)

In [25]:
import pandas as pd
import networkx as nx

class VnExpressBasic:
    def __init__(self, graph):
        self.graph = graph

    def average_degree(self):
        # Tính trung bình của tổng in_degree và out_degree
        in_degree_sum = sum(dict(self.graph.in_degree()).values())
        out_degree_sum = sum(dict(self.graph.out_degree()).values())
        num_nodes = len(self.graph.nodes())
        return (in_degree_sum + out_degree_sum) / num_nodes

    def graph_density(self):
        # Tính mật độ cho đồ thị có hướng
        num_edges = self.graph.number_of_edges()
        num_nodes = len(self.graph.nodes())
        max_edges = num_nodes * (num_nodes - 1)
        return num_edges / max_edges

    def average_path_length(self):
        # Tính độ dài trung bình của các đường đi trong đồ thị
        try:
            if nx.is_strongly_connected(self.graph):
                return nx.average_shortest_path_length(self.graph)
            else:
                return float('inf')  # Trả về vô hạn nếu đồ thị không mạnh liên thông
        except nx.NetworkXError:
            return float('inf')

    def network_diameter(self):
        # Tính đường kính cho đồ thị có hướng
        try:
            if nx.is_strongly_connected(self.graph):
                return nx.diameter(self.graph)
            else:
                return float('inf')  # Trả về vô hạn nếu đồ thị không mạnh liên thông
        except nx.NetworkXError:
            return float('inf')


# Khởi tạo đối tượng VnExpressBasic và tính các chỉ số
vnexpress = VnExpressBasic(G)

average_degree = vnexpress.average_degree()
graph_density = vnexpress.graph_density()
average_path_length = vnexpress.average_path_length()
network_diameter = vnexpress.network_diameter()

# Tạo DataFrame chứa các chỉ số
data = {
    "Metric": ["Average Degree", "Graph Density", "Average Path Length", "Network Diameter"],
    "Value": [average_degree, graph_density, average_path_length, network_diameter]
}
df = pd.DataFrame(data)

# Lưu DataFrame dưới dạng tệp Excel
file_name = "Average_calculate_directed.xlsx"
df.to_excel(file_name, index=False, engine='openpyxl')

print(f"File saved as {file_name}.")


File saved as Average_calculate_directed.xlsx.


In [6]:
(average_degree, graph_density, average_path_length, network_diameter)

4.88


In [27]:
import networkx as nx
import pandas as pd

# Lớp kế thừa để tính các độ đo trung tâm
class VnExpressCen:
    def __init__(self, graph):
        self.graph = graph

    def in_degree_centrality(self):
        return nx.in_degree_centrality(self.graph)

    def out_degree_centrality(self):
        return nx.out_degree_centrality(self.graph)

    def betweenness_centrality(self):
        return nx.betweenness_centrality(self.graph, normalized=True)

    def closeness_centrality(self):
        return nx.closeness_centrality(self.graph)



# Khởi tạo đối tượng và tính các độ đo trung tâm
vnexpress_cen = VnExpressCen(G)

in_degree_centrality = vnexpress_cen.in_degree_centrality()
out_degree_centrality = vnexpress_cen.out_degree_centrality()
betweenness_centrality = vnexpress_cen.betweenness_centrality()
closeness_centrality = vnexpress_cen.closeness_centrality()

# Tạo DataFrame từ các độ đo trung tâm
df_centrality = pd.DataFrame({
    "Node": list(in_degree_centrality.keys()),
    "In-Degree Centrality": list(in_degree_centrality.values()),
    "Out-Degree Centrality": list(out_degree_centrality.values()),
    "Betweenness Centrality": list(betweenness_centrality.values()),
    "Closeness Centrality": list(closeness_centrality.values()),
})

# Lưu DataFrame dưới dạng tệp Excel
file_name_centrality = "Node_Centralities_Directed.xlsx"
df_centrality.to_excel(file_name_centrality, index=False, engine='openpyxl')

print(f"File saved as {file_name_centrality}.")


File saved as Node_Centralities_Directed.xlsx.


In [37]:
!pip install leidenalg igraph pandas networkx


In [38]:
import networkx as nx
import igraph as ig
import leidenalg as la
import pandas as pd

# Đọc đồ thị từ tệp GraphML
graph_file_path = 'social_network.graphml'  # Đường dẫn tệp GraphML của bạn
G_nx = nx.read_graphml(graph_file_path)

# Chuyển đổi đồ thị có hướng của NetworkX thành đồ thị không hướng của igraph
G_igraph = ig.Graph.from_networkx(G_nx.to_undirected())

# Chạy thuật toán Leiden để phân cộng đồng
partition = la.find_partition(G_igraph, la.ModularityVertexPartition)

# Lấy kết quả phân cụm (cộng đồng) từ thuật toán Leiden
community_labels = partition.membership

# Lưu kết quả phân cụm vào DataFrame
df = pd.DataFrame(list(G_nx.nodes(data=True)))  # Lấy thông tin của các node
df['community'] = community_labels  # Thêm cột cộng đồng

# Lưu kết quả vào tệp Excel
output_file_path = 'leiden_results.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Đã lưu kết quả phân cộng đồng vào {output_file_path}")


Đã lưu kết quả phân cộng đồng vào leiden_results.xlsx


In [39]:
num_communities = len(set(community_labels))
print(f"Số lượng cộng đồng: {num_communities}")

# In ra giá trị modularity
modularity_value = partition.modularity
print(f"Giá trị modularity: {modularity_value:.4f}")

Số lượng cộng đồng: 94
Giá trị modularity: 0.5224
